In [1]:
import cobra
from cobra import Reaction
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import ast

# Save JSON model with pythonified IDs and subsystem information

In [2]:
M = cobra.io.load_matlab_model('Recon3DModel_301.mat') # takes several minutes

# fix reaction and metabolite IDs
for rxn in M.reactions:
    rxn.id = rxn.id.replace('[','_').replace(']','')

M.repair()

for m in M.metabolites:
    m.id = m.id.replace('[','_').replace(']','')

M.repair()

# add ATPM reaction
ATPM = Reaction('ATPM')
M.add_reaction(ATPM)
M.reactions.ATPM.lower_bound = 0
M.reactions.ATPM.add_metabolites({'atp_c':-1.0,'adp_c':1.0,'pi_c':1.0})
M.repair()

# set objective to full biomass
M.reactions.biomass_maintenance.objective_coefficient = 0
M.reactions.biomass_reaction.objective_coefficient = 1

# set compartment properties
for m in M.metabolites:
    m.compartment = m.id.split('_')[-1]

# Add subsystem information
rxn_ss = pd.read_csv('./subsystem_data/Recon3_rxn_subsystem.csv',index_col=0).set_index('rxn ID')

for rxn in rxn_ss.index:
    rxn_updated_id = rxn.replace('[','_').replace(']','')
    
    ss = ast.literal_eval(rxn_ss.loc[rxn]['subSystem'])
    
    M.reactions.get_by_id(rxn_updated_id).subsystem = ss
    
# save
cobra.io.save_json_model(M, 'Recon3DModel_301_pythonified.json')

# Reduce medium + exits
Set up a simple medium of glucose, oxygen, essential amino acids and a couple of other metabolites. Block all sinks and DM reactions. 

In [3]:
model = M.copy()

# Block sink and demand reactions
for rxn in model.reactions:
    if 'sink_' in rxn.id or 'DM_' in rxn.id:
        rxn.upper_bound = 0
        rxn.lower_bound = 0
        
# set very simple medium that allows growth
for rxn in model.reactions:
    if 'EX_' in rxn.id:
        rxn.upper_bound = 0 # block all outward
        rxn.lower_bound = 0
        
for rxn in ['EX_h2o_e','EX_h_e','EX_co2_e','EX_nh4_e','EX_pi_e','EX_hco3_e','EX_so4_e','EX_o2_e']:
    model.reactions.get_by_id(rxn).lower_bound = - 1000

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e',
            'EX_glu_L_e', 'EX_gln_L_e', 'EX_tag_hs_e'
           ]:
    model.reactions.get_by_id(rxn).lower_bound = -0.1
    
model.reactions.EX_glc_D_e.lower_bound = -1
        
for rxn in ['EX_h2o_e','EX_h_e','EX_nh4_e','EX_pi_e','EX_so4_e',
           'EX_co2_e','EX_lac_D_e','EX_lac_L_e',
           'EX_urea_e','EX_acac_e','EX_bhb_e','EX_ppa_e','EX_cyst_L_e' ]: # 
    model.reactions.get_by_id(rxn).upper_bound = 1000
    

# Export to JSON    
cobra.io.save_json_model(model, 'Recon3DModel_301_simple_medium.json')

sol = cobra.flux_analysis.pfba(model)
print('After changes',sol.fluxes['biomass_reaction'])
model.summary()

After changes 0.1688863968762772
IN FLUXES          OUT FLUXES     OBJECTIVES
-----------------  -------------  ----------------------
o2_e      2.87     h2o_e  3.37    biomass_reac...  0.169
glc_D_e   1        co2_e  2.62
nh4_e     0.426    h_e    0.89
pi_e      0.21     bhb_e  0.64
gln_L_e   0.1      ppa_e  0.0742
glu_L_e   0.1      so4_e  0.0663
his_L_e   0.1
lys_L_e   0.1
met_L_e   0.1
val_L_e   0.1
leu_L_e   0.0921
phe_L_e   0.0708
thr_L_e   0.0528
ile_L_e   0.0483
tag_hs_e  0.0463
trp_L_e   0.00225


In [ ]:
# keep M unchanged
M = model.copy()
model = M.copy()

# Check oxygen dependence

In [ ]:
model.reactions.EX_o2_e.lower_bound = 0
print(model.optimize())

# model.metabolites.get_by_id('o2_c').summary()

## How is ATP generated? 
### When maximizing ATPM using only glucose and o2

In [ ]:
model = M.copy()

for rxn in ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e','EX_thr_L_e','EX_trp_L_e','EX_val_L_e',
           'EX_glu_L_e','EX_gln_L_e']:
    model.reactions.get_by_id(rxn).lower_bound = 0

model.objective = model.reactions.ATPM
model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()

### When maximizing biomass

In [ ]:
model = M.copy()

model.optimize()

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()
print()
model.metabolites.icit_m.summary()

## How does the model behave under oxygen limitation? 

In [ ]:
model = M.copy()

model.reactions.EX_o2_e.lower_bound = -0.1
sol = cobra.flux_analysis.pfba(model)

model.summary()
print()
model.metabolites.atp_c.summary()
print()
model.metabolites.atp_m.summary()